# Basic

In [391]:
import numpy as np
from numpy import inf

In [458]:
def value_bin(S, K, r, q, sigma, T, M, n, Save, pass_t, contract, space="Linear", method="S"):
    dt = T / n
    u = np.exp(sigma * (dt ** 0.5))
    d = np.exp(-sigma * (dt ** 0.5))
    prob = (np.exp((r - q) * dt) - d) / (u - d)
    tree = []
    for i in range(n+1):
        p_t = []
        for j in range(i+1):
            A_list = A(i, j, S, u, d, M, Save, pass_t, dt, space)
            V_list = np.maximum(A_list - K, 0)
            p_t.append([S * (u ** (i-j)) * (d ** j), 
                        A_list,
                        V_list
                       ])
        tree.append(p_t)
    value = back_induction(tree, S, prob, u, d, r, dt, contract, method)
    return value

In [459]:
def back_induction(p, S, prob, u, d, r, dt, contract, method):
    for i in range(len(p) - 1):
        l = p[len(p) - i - 2] # layer
        l_next = p[len(p) - i - 1]
        for j in range(len(l)):
            l_u = l_next[j][1]
            l_d = l_next[j+1][1]
            Au = ((len(p) - 2 - i + 1) * l[j][1] + S * (u ** (len(p) - 2 - i - j + 1)) * (d ** (j))) / (len(p) - 2 - i + 2)
            Ad = ((len(p) - 2 - i + 1) * l[j][1] + S * (u ** (len(p) - 2 - i - j)) * (d ** (j + 1))) / (len(p) - 2 - i + 2)
            u_i, u_i1, d_i, d_i1, u_w, d_w = find_index(Au, Ad, l_u, l_d, method)
            Cu = u_w * l_next[j][-1][u_i] + (1 - u_w) * l_next[j][-1][u_i1]
            Cd = d_w * l_next[j+1][-1][d_i] + (1 - d_w) * l_next[j+1][-1][d_i1]
            if contract == "E":
                l[j].append((prob * Cu + (1 - prob) * Cd) * np.exp(-r * dt)) 
            else:
                l[j].append(np.maximum(l[j][2], prob * Cu + (1 - prob) * Cd) * np.exp(-r * dt))
        p[len(p) - i - 2] = l
    return p

In [460]:
def A(i, j, S, u, d, M, Save, pass_t, dt, space="Linear"):
    Amax = (((1 - u ** (i - j + 1)) / (1 - u)) + (u ** (i - j)) * d * ((1 - d ** j) / (1 - d))) * S / (i + 1)
    Amin = (((1 - d ** (j + 1)) / (1 - d)) + (d ** j) * u * ((1 - u ** (i - j)) / (1 - u))) * S / (i + 1)
    if pass_t != 0:
        Amax = (dt * i * Amax + pass_t * Save) / (pass_t + dt * i)
        Amin = (dt * i * Amin + pass_t * Save) / (pass_t + dt * i)
        if space == "Linear":
            return np.linspace(Amax, Amin, M+1) # linear
        else:
            return np.exp(np.linspace(np.log(Amin), np.log(Amax), M+1)) # log   
    else:
        if space == "Linear":
            return np.linspace(Amax, Amin, M+1) # linear
        else:
            return np.exp(np.linspace(np.log(Amin), np.log(Amax), M+1)) # log   

In [461]:
def find_index(Au, Ad, A_u, A_d, method="S"):
    u_i = []
    u_i1 = []
    d_i = []
    d_i1 = []
    u_w = []
    d_w = []
    if method == "S":
        for i in range(len(Au)):
            u1 = 0
            u = len(A_u) - 1
            while (Au[i] < A_u[u1]) and u1 < len(Au)-1:
                u1 = u1 + 1
            u_i1.append(u1)
            while (Au[i] > A_u[u]) and u > 0:
                u = u - 1
            u_i.append(u)
        for j in range(len(Au)):
            d1 = 0
            d = len(A_d) - 1
            while (Ad[j] < A_d[d1]) and d1 < len(Ad)-1:
                d1 = d1 + 1
            d_i1.append(d1)
            while (Ad[j] > A_d[d]) and d > 0:
                d = d - 1
            d_i.append(d)
        u_w = (A_u[u_i] != A_u[u_i1]) * (Au - A_u[u_i1]) / (A_u[u_i] - A_u[u_i1])
        d_w = (A_d[d_i] != A_d[d_i1]) * (Ad - A_d[d_i1]) / (A_d[d_i] - A_d[d_i1])
        u_w[np.abs(u_w) > 1] = 1
        d_w[np.abs(d_w) > 1] = 1     
        u_w = np.nan_to_num(u_w)
        d_w = np.nan_to_num(d_w)
    elif method == "B":
        for i in range(len(Au)):
            u_i.append(binary_search(Au[i], A_u)[0])
            u_i1.append(binary_search(Au[i], A_u)[1])
        for j in range(len(Ad)):
            d_i.append(binary_search(Ad[i], A_d)[0])
            d_i1.append(binary_search(Ad[i], A_d)[1])
        u_w = (A_u[u_i] != A_u[u_i1]) * (Au - A_u[u_i1]) / (A_u[u_i] - A_u[u_i1])
        d_w = (A_d[d_i] != A_d[d_i1]) * (Ad - A_d[d_i1]) / (A_d[d_i] - A_d[d_i1])
        u_w[np.abs(u_w) > 1] = 1
        d_w[np.abs(d_w) > 1] = 1     
        u_w = np.nan_to_num(u_w)
        d_w = np.nan_to_num(d_w)
    return u_i, u_i1, d_i, d_i1, u_w, d_w           

In [464]:
value_bin(50, 50, 0.1, 0.05, 0.8, 0.25, 100, 100, 50, 0, "E")[0][0][3][0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


4.735385345731978

In [465]:
value_bin(50, 50, 0.1, 0.05, 0.8, 0.25, 100, 100, 50, 0, "A")[0][0][3][0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


5.413837293578237

In [466]:
value_bin(50, 50, 0.1, 0.05, 0.8, 0.25, 100, 100, 50, 0.25, "E")[0][0][3][0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


4.444215103696563

In [467]:
value_bin(50, 50, 0.1, 0.05, 0.8, 0.25, 100, 100, 50, 0.25, "A")[0][0][3][0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


4.901273830597896

In [468]:
def value_mc(S, K, r, q, sigma, T, M, n, Save, pass_t, num, repeat):
    dt = T / n
    v_list = []
    S_ = np.zeros([repeat, num])
    for j in range(n - 1):
        S_list = np.exp(np.log(S) + (r - q - 0.5 * sigma ** 2) * (j) * dt + \
                        np.random.randn(repeat, num) * sigma * (((j) * dt) ** 0.5))
        S_ += S_list
    V_ = S_ / (n + 1) - K
    V_ = V_ * (V_ > 0)
    value = np.maximum(np.amax(V_, axis=1), 0) * np.exp(-r * T)
    return value.mean(), [value.mean() - 2 * value.std(), value.mean() + 2 * value.std()]

In [469]:
value_mc(50, 50, 0.1, 0.05, 0.8, 0.25, 100, 100, 50, 0, 10000, 20)

(5.215292025801697, [4.136153020207686, 6.294431031395707])

## Bonus1

In [433]:
linear = []
logarith = []
for i in range(10):
    linear.append(value_bin(50, 50, 0.1, 0.05, 0.8, 0.25, 20*(i+1), 100, 50, 0, "E")[0][0][3][0])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


KeyboardInterrupt: 

In [ ]:
for i in range(10):
    logarith.append(value_bin(50, 50, 0.1, 0.05, 0.8, 0.25, 20*(i+1), 100, 50, 0, "E", "Log")[0][0][3][0])

## Bonus2

In [447]:
def binary_search(x, arr):
    start = 0
    end = int(len(arr) - 1)
    while (end - start) > 1:
        if x == arr[int(np.floor((start + end) / 2))]:
            return int(np.floor((start + end) / 2)), int(np.floor((start + end) / 2))
        elif x > arr[int(np.floor((start + end) / 2))]:
            start = int(np.floor((start + end) / 2))
        else:
            end = int(np.floor((start + end) / 2))
    return start, end

In [470]:
value_bin(50, 50, 0.1, 0.05, 0.8, 0.25, 100, 100, 50, 0, "E", "B")[0][0][3][0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


39.63771133635002